In [1]:
import S2VT
import readin_raw as read
import torch
import numpy as np

In [2]:
input_size = 4096
batch_size = 64
encoder_hidden = 256
decoder_hidden = 256
encoder_layer = 1
decoder_layer = 1

In [3]:
training_data, one_hot_size, answer_len = read.generate_dataloader(batch_size = batch_size)

Loading Data
1450
Finish count words
Process data  0  to list
Process data  1  to list
Process data  2  to list
Process data  3  to list
Process data  4  to list
Process data  5  to list
Process data  6  to list
Process data  7  to list
Process data  8  to list
Process data  9  to list
Process data  10  to list
Process data  11  to list
Process data  12  to list
Process data  13  to list
Process data  14  to list
Process data  15  to list
Process data  16  to list
Process data  17  to list
Process data  18  to list
Process data  19  to list
Process data  20  to list
Process data  21  to list
Process data  22  to list
Process data  23  to list
Process data  24  to list
Process data  25  to list
Process data  26  to list
Process data  27  to list
Process data  28  to list
Process data  29  to list
Process data  30  to list
Process data  31  to list
Process data  32  to list
Process data  33  to list
Process data  34  to list
Process data  35  to list
Process data  36  to list
Process dat

In [7]:


output_dim = one_hot_size

learning_rate = 0.001
training_epoch = 100
cuda_avilabel = True
# we can change the schedule sampling rate by parameter lamda; the first 100 epochs set the rate to 1
lamda = 0.999

In [5]:
def training(input_size,batch_size,encoder_hidden,decoder_hidden,encoder_layer,decoder_layer,output_dim,cuda_avilabel,lamda,training_data,load_model = "", optim_state = ""):
    model = None
    if load_model == "":
        model = S2VT.S2VT(input_size,batch_size,encoder_hidden,decoder_hidden,encoder_layer,decoder_layer,output_dim, True)
    else:
        model = torch.load(load_model)
    if(cuda_avilabel):
        model = model.cuda()
    print(model)
    optimizer = None
    if optim_state == "":
        optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
    else:
        state = torch.load(optim_state)
        optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
        optimizer.load_state_dict(state)
    criterion = torch.nn.CrossEntropyLoss()
    loss_array = []
    optimizer.zero_grad()
    
    for epoch in range(training_epoch):
        print("epoch: ",epoch)
        epoch_loss = 0
        for x,y,z in training_data:
            encoder_input = x  # the input to encoder during encoding stage
            correct_answer = y   # correct answer 
	        #encoder_pad = torch.zeros((correct_answer[0],correct_answer[1],input_size),dtype = torch.float)      # the input to encoder during decoding stage
            _, target = correct_answer.max(dim=2)  # not one-hot, but a number imply the right class
            
            if(cuda_avilabel):
                encoder_input = encoder_input.transpose(1, 0).cuda()
                correct_answer = correct_answer.transpose(1, 0).cuda()
	        	#encoder_pad = encoder_pad.cuda()
                target = target.cuda()
            if correct_answer.shape[1] != batch_size:
                model.encoder_h = torch.zeros((encoder_layer, correct_answer.shape[1], encoder_hidden),
                                      dtype = torch.float32).cuda()
                model.encoder_c = torch.zeros((encoder_layer, correct_answer.shape[1], encoder_hidden),
                                              dtype = torch.float32).cuda()
                model.decoder_h = torch.zeros((decoder_layer, correct_answer.shape[1], decoder_hidden),
                                              dtype = torch.float32).cuda()
                model.decoder_c = torch.zeros((decoder_layer, correct_answer.shape[1], decoder_hidden),
                                              dtype = torch.float32).cuda()
                model.batch_size = correct_answer.shape[1]
            else:
                model.encoder_h = torch.zeros((encoder_layer, batch_size, encoder_hidden),
                                      dtype = torch.float32).cuda()
                model.encoder_c = torch.zeros((encoder_layer, batch_size, encoder_hidden),
                                              dtype = torch.float32).cuda()
                model.decoder_h = torch.zeros((decoder_layer, batch_size, decoder_hidden),
                                              dtype = torch.float32).cuda()
                model.decoder_c = torch.zeros((decoder_layer, batch_size, decoder_hidden),
                                              dtype = torch.float32).cuda()
                model.batch_size = batch_size
	        # target = test_correct_answer[iteration] # not one-hot, but a number imply the right class
            output = model(encoder_input,correct_answer, answer_len, z) # correct_answer is just a parameter but not used during encoding stage
	        # eocoded can be used for attention
	        #model.encoder_stage = False
	        #model.decoder_stage = True
	        #output = model(encoder_pad,correct_answer)
            #print(target[0,:z[0].item()])
            
            
            loss = None
            output_tmp = torch.stack(output) #(seq_len,batch,one_hot)
            #print(len(output))
            #print(output_tmp.shape)
            #print(output_tmp[:z[0].item(),0].shape)
            #break
            
            for data_num in range(z.shape[0]):
                length = z[data_num].item()
                if type(loss) == type(None):
                    loss = criterion(output_tmp[:length,data_num], target[data_num,:length])
                else :
                    loss += criterion(output_tmp[:length,data_num], target[data_num,:length])
            '''
            output_tmp = torch.cat((output),0)
            output = output_tmp.view(-1,output_dim)
            target = target.view(-1)
            loss = None
            position = 0
            for data_num in range(z.shape[0]):
                length = z[data_num].item()
                if type(loss) == type(None):
                    loss = criterion(output_tmp[position:position+length], target[position:position+length])
                else :
                    loss += criterion(output_tmp[position:position+length], target[position:position+length])
                position += answer_len # answer_len is the max length for a sentence
            '''
            
            '''
            print(output_tmp[0][:9].shape)
            output = output_tmp.view(-1,output_dim)
            print(output.shape)
            target = target.view(-1)
            loss = criterion(output,target)
            '''
            #print(loss.item())
            epoch_loss += loss.item()
            loss_array.append(loss.item())
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(epoch_loss / (1450//64 + 1))
        # we can change the schedule sampling rate by parameter lamda; the first 100 epochs set the rate to 1
        if epoch >= 50:
            tmp_rate = model.schedule_sample_rate
            model.schedule_sample_rate = lamda * tmp_rate
    ### save model ###
    torch.save(model,'s2vt_1025_2.pkl')
    torch.save(optimizer.state_dict(), "s2vt_1025_optim.pkl")
    return loss_array

def restore_model(model_name):
	### restore model ###
	net = torch.load(model_name)
	print(net)
	return net

In [ ]:

history = training(input_size=input_size, batch_size=batch_size, encoder_hidden=encoder_hidden,
                   decoder_hidden=decoder_hidden, encoder_layer=encoder_layer, decoder_layer=decoder_layer,
                   output_dim=one_hot_size, cuda_avilabel=True, lamda=lamda, training_data = training_data)

for i in range(2):
    training_data, one_hot_size, answer_len = read.generate_dataloader(batch_size = batch_size)
    history = training(input_size=input_size, batch_size=batch_size, encoder_hidden=encoder_hidden,
                       decoder_hidden=decoder_hidden, encoder_layer=encoder_layer, decoder_layer=decoder_layer,
                       output_dim=one_hot_size, cuda_avilabel=True, lamda=lamda, training_data = training_data,load_model = "s2vt_1025_2.pkl",optim_state = "s2vt_1025_optim.pkl")



S2VT(
  (output_softmax): Softmax()
  (encoder): LSTM(4096, 256)
  (decoder): LSTM(512, 256)
  (input_embedding): Linear(in_features=2890, out_features=256, bias=True)
  (output_embedding): Linear(in_features=256, out_features=2890, bias=True)
  (attn): Attention(
    (dropout): Dropout(p=0.2)
    (linear_l1du_inputlen): Linear(in_features=512, out_features=80, bias=True)
    (linear_2du_du): Linear(in_features=512, out_features=256, bias=True)
    (softmax): Softmax()
    (relu): PReLU(num_parameters=1)
  )
)
epoch:  0
361.2150122601053
epoch:  1
291.8921800696331
epoch:  2
281.1518514881963
epoch:  3
272.9314581829569
epoch:  4
263.39276720129925
epoch:  5
252.38597637674084
epoch:  6
239.78994551948878
epoch:  7
226.9665805982507
epoch:  8
213.62995578931725
epoch:  9
199.29943549114725
epoch:  10
184.6081075253694
epoch:  11
171.01565452243972
epoch:  12
158.95519289763078
epoch:  13
147.22490891166356
epoch:  14
136.62664894435716
epoch:  15
126.28867240574048
epoch:  16
116.87347

In [ ]:
a=0

In [48]:
for x,y,z in training_data:
    print(z[0])
    break
    
    

tensor([8])


In [9]:
a

1450

In [11]:
answer_len

44

In [33]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Oct 16 09:19:24 2018

@author: jimmy
"""

import torch
import numpy as np

class Attention(torch.nn.Module):
    # Takes necessary information of dimensions of vectors
    # returns a dict of embedding networks
    # b: batch size
    # e_l: layers of encoder
    # e_u: units of encoder, thus hidden of encoder = (e_l, b, e_u) assuming one direction lstm
    # max: maximum sequence length, defaults to 1024
    # p: word size, thus prediction of decoder at one time step = (b, p)
    def __init__ (self, e_l, e_u, d_l, d_u, b, max, p):
        super(Attention, self).__init__()

        self.dropout = torch.nn.Dropout(0.2)
        self.linear_l1du_max = torch.nn.Linear((d_l + 1) * d_u, max)
        self.linear_2du_du = torch.nn.Linear(2 * d_u, d_u)
        self.softmax = torch.nn.Softmax(dim=1)
        self.relu = torch.nn.PReLU(num_parameters=d_l)

    def concat(self, x, y):
        # concatenate two tensors along last axis (which is usually num of units)
        assert len(x.shape) == len(y.shape), 'Attention.concat(): no matching tensor ranks.'
        return torch.cat((x, y), -1)

    def bvm(self, BS, SBH):
        # eliminate S from both tensors, returns LBH
        B1S = BS.unsqueeze(1)
        BSH = SBH.transpose(0, 1)
        B1H = torch.bmm(B1S, BSH)
        return B1H.squeeze(1)

    def forward(self, h, word, E):
        # h: (layer, batch, units)
        # word: (1, batch, units)
        # E: (seq_max or input_seq_len, batch, units)
        assert len(h.shape) == 3, 'Attention.forward(): hidden state vector is of rank {}, not 3.'.format(len(h.shape))
        assert word.shape[0], 'Attention.forward(): sequence of words (len={}) are not accepted.'.format(word.shape[0])

        i = h[0]
        for _ in range (h.shape[0] - 1):
            i = self.concat(i, h[_ + 1])
        a = self.concat(i, word.squeeze(0))         # (batch, (layer + 1)*units)
        w = self.softmax(self.linear_l1du_max(a))   # (batch, max)
        f = self.bvm(w, E)                          # (batch, units)
        c = self.concat(word.squeeze(0), f)         # (batch, 2*units)
        o = self.relu(self.linear_2du_du(c))        # (batch, units)
        return o.unsqueeze(0)                       # (1, batch, units)


class S2VT(torch.nn.Module):
    def __init__(self, input_dim, batch, encoder_unit, decoder_unit,
                 encoder_layer, decoder_layer, one_hot, use_attention):
        super(S2VT, self).__init__()

        ### PARAMETERS ###
        #Need to add embedding size
        self.one_hot_size = one_hot
        self.batch_size = batch
        self.encoder_h_size = encoder_unit
        self.decoder_h_size = decoder_unit
        self.encoder_input_size = input_dim
        self.decoder_input_size = (decoder_unit + encoder_unit)
        self.encoder_layer = encoder_layer
        self.decoder_layer = decoder_layer
        self.output_softmax = torch.nn.Softmax(dim=1)
        self.use_attention = use_attention

        #Need to change this ratio by model.schedule_sample_rate
        self.schedule_sample_rate = 1

        ### SUBMODULES ###
        # shape = (num_layer, batch, hidden_size)
        self.encoder_h = torch.zeros((encoder_layer, batch, encoder_unit),
                                      dtype = torch.float32).cuda()
        self.encoder_c = torch.zeros((encoder_layer, batch, encoder_unit),
                                      dtype = torch.float32).cuda()
        self.decoder_h = torch.zeros((decoder_layer, batch, decoder_unit),
                                      dtype = torch.float32).cuda()
        self.decoder_c = torch.zeros((decoder_layer, batch, decoder_unit),
                                      dtype = torch.float32).cuda()

        self.encoder = torch.nn.LSTM(input_size = self.encoder_input_size,
                                     hidden_size = encoder_unit,
                                     num_layers = encoder_layer,
                                     dropout = 0)
        self.decoder = torch.nn.LSTM(input_size = self.decoder_input_size,
                                     hidden_size = decoder_unit,
                                     num_layers = decoder_layer,
                                     dropout = 0)

        self.input_embedding = torch.nn.Linear(self.one_hot_size, self.decoder_h_size)
        self.output_embedding = torch.nn.Linear(self.decoder_h_size, self.one_hot_size)

        self.attn = Attention(e_l=encoder_layer,
                              e_u=encoder_unit,
                              d_l=decoder_layer,
                              d_u=decoder_unit,
                              b=batch,
                              max=80,
                              p=one_hot)

    def forward(self, input_data, correct_answer, max_len, real_ans_length):
        # correct_answer : whole sequence of answer
        # input_data shape: (sequence_len, batch, input_size)
        # encoded_sequence shape: (sequence_len, batch, input_size)
        # First the encoder is fed with input sequence, then
        # the decoder is fed with one word from the result
        # sequence of encoder at a step.
        # When the encoder is receiving the sequence, the decoder also
        # receives its output; when the decoder is decoding, the encoder
        # does nothing.

        #print("Encoding")
        encoded_sequence, (encoder_hn, encoder_cn) = \
            self.encoder(input_data, (self.encoder_h, self.encoder_c))

        padding_tag = torch.zeros((len(input_data), self.batch_size, self.decoder_h_size),
                                  dtype=torch.float32).cuda()

        decoder_input = torch.cat((encoded_sequence, padding_tag), dim = 2)
        decoder_output, (decoder_hn, decoder_cn) = \
            self.decoder(decoder_input, (self.decoder_h, self.decoder_c))

        #print("Decoding")
        decoder_output_words = []
        bos_tag = torch.zeros((1, self.batch_size, self.one_hot_size),
                                  dtype= torch.float32).cuda()
        bos_tag[:,:,-2] = 1
        padding_input = torch.zeros((max_len, self.batch_size, self.encoder_input_size),
                                    dtype=torch.float32).cuda()

        encoded_padding, (encoder_hn, encoder_cn) = self.encoder(padding_input, (encoder_hn, encoder_cn))

        for time in range(max_len):
            context = None
            sample = None
            if time == 0:
                # sample a <bos> at first time step
                bos_embedding = self.input_embedding(bos_tag)
                sample = bos_embedding
                context = self.attn(decoder_hn, sample, encoded_sequence)  + (encoded_padding[time]).unsqueeze(0) if self.use_attention \
                    else  (encoded_padding[time]).unsqueeze(0)

            else:
                #some change by Wei-Tsung
                # time > 0
                # randomly choose previous prediction or correct answer as input
                sample = self.input_embedding(correct_answer[time].unsqueeze(0)) \
                    if np.random.random() < self.schedule_sample_rate \
                    else decoder_output

                # process sample with attention
                context = self.attn(decoder_hn, sample, encoded_sequence) + (encoded_padding[time]).unsqueeze(0) if self.use_attention \
                    else (encoded_padding[time]).unsqueeze(0)

            decoder_input = torch.cat((sample, context), dim = 2)
            decoder_output, (decoder_hn, decoder_cn) = \
                self.decoder(decoder_input, (decoder_hn, decoder_cn))

            word = self.output_embedding(decoder_output).squeeze(0)
            
            # word is of shape (batch_size, one_hot_size)
            decoder_output_words.append(word)
            
        return decoder_output_words
    
    def test(self, input_data, max_len):
        # input_data shape: (sequence_len, batch, input_size)
        # encoded_sequence shape: (sequence_len, batch, input_size)
        # First the encoder is fed with input sequence, then
        # the decoder is fed with one word from the result
        # sequence of encoder at a step.
        # When the encoder is receiving the sequence, the decoder also
        # receives its output; when the decoder is decoding, the encoder
        # does nothing.

        print("Encoding")
        encoded_sequence, (encoder_hn, encoder_cn) = \
            self.encoder(input_data, (self.encoder_h, self.encoder_c))

        padding_tag = torch.zeros((len(input_data), self.batch_size, self.decoder_h_size),
                                  dtype=torch.float32).cuda()

        decoder_input = torch.cat((encoded_sequence, padding_tag), dim = 2)
        decoder_output, (decoder_hn, decoder_cn) = \
            self.decoder(decoder_input, (self.decoder_h, self.decoder_c))

        print("Decoding")
        decoder_output_words = []
        bos_tag = torch.zeros((1, self.batch_size, self.one_hot_size),
                                  dtype= torch.float32).cuda()
        bos_tag[:,:,-2] = 1
        padding_input = torch.zeros((max_len, self.batch_size, self.encoder_input_size),
                                    dtype=torch.float32).cuda()

        encoded_padding, (encoder_hn, encoder_cn) = self.encoder(padding_input, (encoder_hn, encoder_cn))

        for time in range(max_len):
            context = None
            sample = None
            if time == 0:
                # sample a <bos> at first time step
                bos_embedding = self.input_embedding(bos_tag)
                sample = bos_embedding
                context = self.attn(decoder_hn, sample, encoded_sequence)  + (encoded_padding[time]).unsqueeze(0) if self.use_attention \
                    else  (encoded_padding[time]).unsqueeze(0)

            else:
                sample = decoder_output

                # process sample with attention
                context = self.attn(decoder_hn, sample, encoded_sequence) + (encoded_padding[time]).unsqueeze(0) if self.use_attention \
                    else (encoded_padding[time]).unsqueeze(0)

            decoder_input = torch.cat((sample, context), dim = 2)
            decoder_output, (decoder_h, decoder_c) = \
                self.decoder(decoder_input, (decoder_hn, decoder_cn))

            word = self.output_embedding(decoder_output).squeeze(0)
            word = self.output_softmax(word)
            # word is of shape (batch_size, one_hot_size)
            decoder_output_words.append(word)
            
        return decoder_output_words